# 22: Flow and functions in Python

Author: Greg Wray  
Date: 2025-MAR-18
  
Type code into this notebook during lecture. Run code in the selected cell by clicking the `run` (play button) icon or typing `shift-return`.  
Modify and experiment!! This is the best way to get a feel for how Python works (and any other language).   
Consider adding comments to record notes and findings: `#` starts a comment on a new line or part way through a line (just like R and bash). 

## Example of functions and control flow

The following program converts a protein FASTA file into a more easily computable format. This program illustrates the use of functions and control flow, as well as working with dictionaries and using context managers for file operations.

FASTA files have an awkward organization, with each entry on two consecutive lines; in addition, there may or may not be a description following the file identifier and the sequence may or may not contain newline escape sequences. 

This program reads a protein FASTA file and converts the contents into a Python dictionary with the following organization:      
* key = identifier with '>' removed   
* value = list composed of two items: the description after the identifier plus the sequence with newlines removed   

It includes three functions to work with the dictionary: 
* view a specified range of items 
* add a "column" representing the length of the protein
* output the dictionary in `.csv` fomat

In [1]:
### program to read a FASTA file and convert to dictionary 
# and with ability to ouput dictionary as a .csv file

# function to create a new "column" containing the length of the protein
def add_length(dict):
    for curr_key, curr_val in dict.items():
        length = len(curr_val[1])
        dict[curr_key].append(length)
    return dict

# function to return a specified range of items in a dictionary as a list
def view_dict(dict, start=0, stop=10):
    items = list(dict.items())[start:stop]
    return items

# function to convert the dictionary into a .csv file
def to_csv(dict, fname):
    with open(fname, 'w') as outfile:
        header = 'ID,description,sequence,length\n'
        outfile.write(header)
        for curr_key, curr_val in dict.items():
            curr_val[0] = curr_val[0].replace(',', '+')
            curr_val[2] = str(curr_val[2])
            next_line = curr_key + ',' + ",".join(curr_val) + '\n'
            outfile.write(next_line)
    print('wrote data to', fname)
    
# read contents of file into a single string
with open('short.fasta', 'r') as infile:
    contents = infile.read()

# create a list of entries
entries = contents.split('>')
entries = entries[1:]

# convert the entries into a dictionary
protein_dict = {}
for i in entries:
    id = i[0:14]
    i = i[14:]
    j = i.find('\n')
    desc = i[:j].strip()
    seq = i[j:].strip()
    seq = seq.replace('\n', '')
    protein_dict[id] = [desc, seq]

# create a new "column" containing the length of the protein
protein_dict_2 = add_length(protein_dict)

#print(new_dict)

#view a range of items in the dictionary
#items = view_dict(protein_dict_2, 2, 5)
#print(items)

# export the dictionary to a file
to_csv(protein_dict_2, 'protfile.csv')


wrote data to protfile.csv


## Functional programming with map() and filter()
`map()` and `filter()` provide a compact and readable way to apply an operation to every item in an iterable by creating **implicit loops**. These functions are useful in situations where you want to apply *simple* functions or filtering operations. Note that the first argument passed to these these functions is a function  and the second argument is an interable.  

In [27]:
# create some lists to work with
list_a = ['armadillo', 'orca', 'three-toed sloth', 'pronghorn', 'aardvark', 'pangolin', 'fruit bat']
list_b = list(range(20))
print(list_b)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


Use `map()` to apply a function to each item in an iterable. `map()` takes two arguments: the function you want to apply and an iterable that you want to apply it to. Note that the function must take exactly one argument, namely the next item in the iterable. Also note that `map()` returns a map object, not a data object of the same type you give it. For this reason, it is common to wrap calls to `map()` with `list()` so that you can work with the result. 

In [28]:
# find the length of each item the traditional way using a loop
list_c = []
for i in list_a:
    list_c.append(len(i))
list_c

[9, 4, 16, 9, 8, 8, 9]

In [29]:
# apply map() to a list of strings
list_d = list(map(len, list_a))
list_d

[9, 4, 16, 9, 8, 8, 9]

In [24]:
# find the square of each item the traditional way using a loop
list_e = []
for i in list_b:
    list_e.append(i**i)
list_e

[1,
 1,
 4,
 27,
 256,
 3125,
 46656,
 823543,
 16777216,
 387420489,
 10000000000,
 285311670611,
 8916100448256,
 302875106592253,
 11112006825558016,
 437893890380859375,
 18446744073709551616,
 827240261886336764177,
 39346408075296537575424,
 1978419655660313589123979]

In [25]:
# apply map() to a list of integers
# we first need to define a function
def square(val):
    return val**val
list_f = list(map(square, list_b))
list_f

[1,
 1,
 4,
 27,
 256,
 3125,
 46656,
 823543,
 16777216,
 387420489,
 10000000000,
 285311670611,
 8916100448256,
 302875106592253,
 11112006825558016,
 437893890380859375,
 18446744073709551616,
 827240261886336764177,
 39346408075296537575424,
 1978419655660313589123979]

Use `filter()` to apply a filter to each item in an iterable, returning only those items that match a condition. Importantly, the condition must be specified in a function. The examples below use simple filters, but the filtering criteria can be as complicated as you want because they are encapsulated in the function. Otherwise, `filter()` works similarly to `map()`.

In [26]:
# filter for strings that start with 'p' the traditional way using a loop
list_g = []
for j in list_a:
    if j.startswith('p'): list_g.append(j)
list_g

['pronghorn antelope']

In [30]:
# filter for strings that start with 'p' using filter()
# we first need to define a function
def starts_with_p(val):
    return val.startswith('p')
list_h = list(filter(starts_with_p, list_a))
list_h

['pronghorn', 'pangolin']

In [31]:
# filter for numbers divisible by 3 the traditional way using a loop
list_i = []
for k in list_b:
    if k % 3 == 0: list_i.append(k)
list_i

[0, 3, 6, 9, 12, 15, 18]

In [32]:
# filter for numbers divisible by 3 using filter()
# we first need to define a function
def div_by_3(val):
    return val % 3 == 0
list_j = list(filter(div_by_3, list_b))
list_j

[0, 3, 6, 9, 12, 15, 18]

## Lambda functions
Lambda functions are often described as "anonymous" or "temporary" functions. The examples above illustrate why they are useful. We can simplify and save several lines of code using `map()` or `filter()`; however, in many cases we need to define a function before we can take adavantage of these functions, which somewhat defeats the purpose. Lambda functions solve this problem by allowing us to define a function directly within the call to  `map()` or `filter()`. Lambda functions are also useful in other situations, including *comprehensions* (below). In addition, they are commonly used in Python code, so learning how they work is well worth the effort. In general, lambda functions are most useful when (1) you only need to call the function once and (2) the function is simple. 

In [ ]:
# apply map() to a list of integers using a lambda function
list_k = list(map(lambda x: x**x, list_b))
list_k

In [ ]:
# apply filter() to a list of strings using a lambda function
list_m = list(filter(lambda x: x.startswith('p'), list_a))
list_m

## Functional programming with comprehensions
**Comprehensions** provide another functional programming paradigm in Python. Comprehensions are generally more readable than using `map()` and `filter()`, and they are more versatile because you can combine both operations in a single statement. Comprehensions can be applied to any iterator, but are most commonly applied to lists, so you will likely encounter them in the form of **list comprehensions**.

Here are the basic formulas, applied to lists: 
* apply a function or operation to each item: `new_list = [new_item for item in list]` 
* filter items: `new_list = [item for item in list if condition]`
* apply a function or operation to items after filtering: `new_list = [new_item for item in list if condition]`

As with many programming concepts, it's easiest to see how this works by experimenting with some simple cases. The examples below use comprehensions to carry out the for loop, mapping, and filtering operations covered earlier.

In [ ]:
# use a comprehension to find the length of each item in a list
#    same result as the for loop and the map() function examples above
list_n = [len(x) for x in list_a]
list_n

In [ ]:
# use a comprehension to print out each item in a list and apply a method
[print(x.title()) for x in list_a]

In [ ]:
# use a comprehension to carry out an operation rather than apply a function
list_o = [x + 1 for x in list_b]
list_o

In [ ]:
# use a comprehension to filter for divisible by 3 in a list
#  same result as the for loop and the filer() function examples above
list_p = [x for x in list_b if x % 3 ==0]
list_p

In [ ]:
# use a comprehension to filter find the first letter of strings longer than 5
#    illustrates how to apply a filter and a function in a single statement
list_q = [x[0] for x in list_a if len(x) > 5]
list_q

In the examples above, note that the core of a comprehension is the loop: `for x in list`. To **alter** the result by applying a function or carrying out an operation, change the code to LHS of the loop. To **filter** the items, add a condition to the LHS of the loop. If you only want to alter the result, simply leave out a condition. If you only want to filter, include the loop variable on the LHS without applying a function or operation (as in the third example above). Finally, if you want to do both, they can be combined in a single comprehension (as in the final example above). 

Although comprehensions are most commonly applied to lists, they can be used with any iterable type. Note that you need to wrap the comprehension in the appropriate brackets (square, round, curly) to indicate which type of iterable you want back.

In [ ]:
# example of a comprehension applied to a set rather than a list
#  filters for items of length 6
set_a = {'pink', 'yellow', 'amber', 'indigo', 'gray', 'aqua', 'red', 'green', 'violet'}
set_b = {s for s in set_a if len(s) == 6}
set_b

In [ ]:
# combine filtering with a function for a set comprehension
#    filters for strings of length 6, then extracts the first character
set_c = {s[0] for s in set_a if len(s) == 6}
set_c

The examples so far cover the basic syntax and applications of comprehensions. Below are three useful extensions.
    
First, it is possible to specify multiple conditions in the RHS of a comprehension. It is helpful but not required to organize the conditions onto separate lines for readability. The example below applies three conditions to test for specific starting and ending letters of each string in list. The first condition avoids run-time errors that would arise from trying to access a string of length 0. Note that the conditions are separated by whitespace (space or return), not commas.

In [33]:
# filter based on specific starting and ending letters of a string
list_r = [r for r in list_a
    if len(r) >0
    if r[0] == 'a'
    if r[-1] == 'k'
]
list_r

['aardvark']

Second, it is also possible to use an `if` / `else` structure into the LHS of a comprehension, not to filter, but to perform different operations depending on a condition. A common application is to create a mask for Boolean indexing. (As a quick reminder, we covered Boolean indexing in the first semester with R. This is a fast, versatile way to filter items in a column or other iterable.)

In [ ]:
# classify items in a list of string according to criteria
list_t = [True if len(t) > 8 else False for t in list_a]
list_t

And third, it is possible to nest loops in a comprehension. This is useful for accessing every item in a matrix so that you can apply a function, carry out an operation, filter, or generate a Boolean index. Nesting can also be useful for simply generating a matrix. The example below generates a 5 x 5 matrix of the numbers 0...4. Note the nesting of square brackets to create an inner and outer loop. If you substitute round brackets in one or the other loop, you can generate a tuple of lists or a list of tuples (or any other compund data structure you wish to create). Also note the use of **anonymous variables**, since we do not need to refer to these variable again (the code will generate exactly the same result if you use i and j or some other variable name).  

In [ ]:
# generate a matrix using nested list comprehensions
my_matrix = [[_ for _ in range(5)] for _ in range(5)]
my_matrix

## Exception handlers
Errors that arise during the execution of a program are called **exceptions** in Python. Exceptions can become frustrating if you are running programs that take a long time to execute or that run in an unsupervised setting. The `try` / `except` structure is a special form of control flow that can bypass run-time errors and allow a program to continue excuting. Optionally, custom error reporting can be added. This can also be useful for debugging programs.  

In [ ]:
# handle a divide by zero situation
values = list(range(-3, 4))
for i in values:
    try: 
        print(42/i)
    except:
        print("Warning: divide by zero error!") 
print("Continuing on to remainder of program")

In [ ]:
# trapping specific kinds of errors
values = [-3, -1, 0, 1, 'a', 3]
for i in values:
    try: 
        print(f"Result: {42/i}")
    except ZeroDivisionError as e:
        print(f"Error: {e}")
    except TypeError as t:
        print(f"Error: {t}")
print("Continuing on to remainder of program")

## Custom exceptions
Python provides specific information about the type of error that caused an exception. It can sometimes be useful to create your own exceptions with the `raise` keyword. For example, you may want to trap obviously incorrect temperature measurements by limiting allowable values to a specific range or a specific data type.

In [16]:
# trap obviously incorrect temperature values
values = [22, 24, 19, 3000, 26] 
for t in values:
    if (t < -20) or (t > 110): 
        raise ValueError(f"Temperature unrealistic: {t}")
    else:
        print(f"Temperature = {t}")

Temperature = 22
Temperature = 24
Temperature = 19


ValueError: Temperature unrealistic: 3000

To trap different kinds of errors and provide specific information about each, use one or more `elif` clauses.

In [17]:
# trap obviously incorrect temperature values and non-integer values
values = [22, 24, 19, 'a', 26] 
for t in values:
    if (t < -20) or (t > 110): 
        raise ValueError(f"Temperature unrealistic: {t}")
    elif type(t) == float:
        raise TypeError(f"Value must be an integer: {t}")        
    else:
        print(f"Temperature = {t}")

Temperature = 22
Temperature = 24
Temperature = 19


TypeError: '<' not supported between instances of 'str' and 'int'